# Agent

#### 필수 라이브러리 설치

In [179]:
!pip install -qU langchain langchain-openai langchain_community langchain_experimental langchain-text-splitters tiktoken pypdf faiss-cpu

#### OpenAI API Key

In [167]:
import os

os.environ["OPENAI_API_KEY"] = 'sk-**********************************************************'

## Tools

### Tavily Search

#### tavily serch api key 등록

In [159]:
import os

os.environ["TAVILY_API_KEY"] = "tvly-***********************************************"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [160]:
from langchain_community.tools.tavily_search import TavilySearchResults

#Tavily search 테스트 
#k 는 검색 결과 개수
search = TavilySearchResults(k=5)

In [161]:
# search 결과
search.invoke("양재 시민의 숲에서 가장 가까운 코스트코는 어디인가요?")

[{'url': 'https://m.blog.naver.com/sh2553/223159521355',
  'content': '사이드 메뉴를 다양하게 판매하고 있어 입이 즐겁다. 점심시간에 방문하면 웨이팅이 있을 정도로. 유명하고 맛있는 즉석 떡볶이 맛집이라. 조금 이른시간 혹은 조금 늦게 방문을 추천! 재방문 지수 : ⭐⭐⭐⭐⭐. ⬇️빨강떡볶이 후기 보러가기⬇️. https://blog ...'},
 {'url': 'https://blackbear88.tistory.com/entry/양재양재시민의숲-코스트코-근처-N번째-방문-카레-돈까스-맛집-“아르코”내돈-내산-솔직후기',
  'content': '양재/양재시민의숲 코스트코 근처 맑은 국물이 맛있는 곰탕 맛집 "남도나주곰탕",내돈 내산 솔직후기 (359) 2023.09.22 잠실/잠실새내 롯데타워 고소한 치즈를 활용한 브런치 맛집 "치즈룸&테이스팅룸",내돈 내산 솔직후기 (349)'},
 {'url': 'https://blog.naver.com/PostView.nhn?blogId=abc204308&logNo=223241678694',
  'content': '몽거는 양재시민의 숲에서. 도보로는 15분거리에 위치하고 있어! 주변에 이마트, 코스트코 등 큰 쇼핑센터가 있어서. 마을버스도 다니고 있어! 양재시민의숲역 카페에서 찾기 쉽지않은. 주차가능한 카페라서 차 가져오시는 분들은. 편하게 카페 이용이 가능할 것 ...'},
 {'url': 'https://www.costco.co.kr/store-finder/Yangjae',
  'content': '06772. 서울특별시 서초구 양재대로 159. (양재동 217-6) 전화번호: 1899-9900. 이메일: member@costcokr.com. 길 찾기.'},
 {'url': 'https://moovitapp.com/index/ko/대중_교통-코스트코_양재점-서울시Seoul-site_8477366-1802',
  'content': "코스트코 양재점 까지 가장 

## PDF - Retriever

In [170]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

#load pdf
loader = PyPDFLoader("/content/layout-parser-readthedocs-io-en-latest.pdf")

#text split 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# pdf에 split 적용
split_docs = loader.load_and_split(text_splitter)

#vector db에 데이터 저장
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

#검색기 생성
retriever = vector.as_retriever()

In [173]:
#input값에 대한 유사 정보 검색 테스트 
retriever.get_relevant_documents(
    "layout element에 대해서 설명해줘"
)

[Document(page_content='Layout Parser, Release 0.3.2\n42 Chapter 6. Layout Elements', metadata={'source': '/content/layout-parser-readthedocs-io-en-latest.pdf', 'page': 45}),
 Document(page_content='5.2 Loading and visualizing layouts using Layout-Parser . . . . . . . . . . . . . . . . . . . . . . . . . . 19\n5.3 Model Predictions on loaded data . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 24\n6 Layout Elements 27\n6.1 Coordinate System . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 27\n6.2 TextBlock . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35\n6.3 Layout . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 38\n7 Shape Operations 43\n7.1 Theunion Operation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 44\n7.2 Theintersect Operation . . . . . . . . . . . . . . . . . . . . . . . . 

In [174]:
from langchain.tools.retriever import create_retriever_tool

#검색기를 tool로 변환
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="PDF에 관련된 질문은 이 도구를 사용해야 합니다!",
)

In [175]:
#tool을 리스트로 변환
tools = [search, retriever_tool]

In [176]:
from langchain_openai import ChatOpenAI

#llm 모델 생성
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

#### 필수 라이브러리 설치

In [180]:
!pip install langchainhub

In [181]:
from langchain import hub

#Hub로부터 Prompt 다운로드 
#https://smith.langchain.com/hub?ref=blog.langchain.dev
prompt = hub.pull("hwchase17/openai-functions-agent")

#프롬프트 확인
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

###**Running the agent**

In [182]:
from langchain.agents import create_openai_functions_agent

#OpenAI Function Agent 생성 
agent = create_openai_functions_agent(llm, tools, prompt)

In [184]:
from langchain.agents import AgentExecutor

#Agent 실행기 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [185]:
# agent 실행기 실행 및 답변 출력
response = agent_executor.invoke({"input": "OCR 테이블 파싱하는 방법을 알려주세요"})
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
OCR (Optical Character Recognition)을 사용하여 테이블을 파싱하는 과정은 이미지나 PDF 내의 텍스트를 추출하고, 이를 구조화된 형태로 변환하는 작업을 포함합니다. 테이블 파싱은 특히 데이터가 열과 행으로 구성된 테이블 형태일 때 유용합니다. 다음은 OCR을 사용하여 테이블을 파싱하는 기본적인 단계입니다:

1. **이미지 준비**: 파싱할 문서나 이미지를 준비합니다. 이미지의 품질이 좋을수록 OCR의 정확도가 높아집니다. 필요한 경우 이미지의 명암, 대비, 해상도를 조정하여 품질을 개선할 수 있습니다.

2. **OCR 엔진 선택**: Tesseract, Google Cloud Vision API, Amazon Textract 등 다양한 OCR 엔진 중에서 선택할 수 있습니다. 각 엔진마다 특성과 성능이 다르므로, 요구사항에 맞는 엔진을 선택하는 것이 중요합니다.

3. **텍스트 추출**: 선택한 OCR 엔진을 사용하여 이미지에서 텍스트를 추출합니다. 이 과정에서 엔진은 이미지 내의 문자를 인식하고, 이를 디지털 텍스트로 변환합니다.

4. **테이블 구조 인식**: 추출된 텍스트 중에서 테이블 구조를 인식합니다. 일부 OCR 엔진은 테이블 구조를 자동으로 인식하는 기능을 제공하지만, 그렇지 않은 경우에는 텍스트의 위치 정보를 사용하여 수동으로 테이블 구조를 재구성해야 할 수도 있습니다.

5. **데이터 정제 및 변환**: 테이블 구조를 인식한 후, 데이터를 정제하고 필요한 형식으로 변환합니다. 이 과정에서 불필요한 공백 제거, 열과 행의 정렬, 데이터 형식 변환 등이 포함될 수 있습니다.

6. **데이터 저장 및 활용**: 파싱된 테이블 데이터를 데이터베이스, 스프레드시트, 또는 다른 원하는 형식으로 저장합니다. 저장된 데이터는 분석, 보고서 작성, 데이터 시각화 등 다양한 목적으로 활용할 수 있습니다.

OCR을 사용한 테이블 파싱은 복잡한 과정일 수 

### Agent Memory

In [186]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#채팅 history 생성
message_history = ChatMessageHistory()

In [187]:
#RunnableWithMessageHistory 모듈로 채팅 히스토리 에이전트 생성
#lambda로 message_history 의 번호를 세션값으로 가져옴
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [188]:
# agent 에 대화별 세션 아이디를 부여하여 실행 및 답변 확인
response = agent_with_chat_history.invoke(
    {"input": "Hi! I'm DunnDunn. Glad to meet you."},
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...
Hi DunnDunn! It's great to meet you too. How can I assist you today?

> Finished chain.
답변: Hi DunnDunn! It's great to meet you too. How can I assist you today?


In [193]:
# 한 번 더 테스트
response = agent_with_chat_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...
Your name is DunnDunn. How can I assist you further today?

> Finished chain.
답변: Your name is DunnDunn. How can I assist you further today?


In [192]:
# 동일한 세션아이디로 테스트
response = agent_with_chat_history.invoke(
    {"input": "양재 시민의 숲에서 가장 가까운 코스트코는 어디인가요?"},
     config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '양재 시민의 숲 가장 가까운 코스트코'}`


[{'url': 'https://namu.wiki/w/코스트코 양재점', 'content': '코스트코 옆에는 이마트 양재점, 하나로마트 양재점 하이브랜드, 서울 오토갤러리, nh농협은행, 쌍용차, 더케이 서울호텔까지 있는 혼잡한 도로라서 코스트코 진입 차량 전용 차로까지 있다. 엄청 붐비기 때문에 주말이나 설, 추석 같은 명절 대목에는 바로 ...'}, {'url': 'https://moovitapp.com/index/ko/대중_교통-코스트코_양재점-서울시Seoul-site_8477366-1802', 'content': "코스트코 양재점 까지 가장 가까운 역은 어디인가요? 코스트코 양재점 에 가장 가까운 노선들은 : 양재시민의숲 Yangjae Citizen's Forest 은(는) 611 미터 거리이며, 9 분 도보가 소요됩니다. 양재2동주민센터.양재꽃시장 은(는) 694 미터 거리이며, 10 분 도보가 소요됩니다."}, {'url': 'https://www.costco.co.kr/store-finder/Yangjae', 'content': 'Yangjae/South Korea/853에서 코스트코 쇼핑하기 ... 서울특별시 서초구 양재대로 159 (양재동 217-6) 전화번호: 1899-9900 이메일: member@costcokr.com'}, {'url': 'https://blog.naver.com/PostView.nhn?blogId=abc204308&logNo=223241678694', 'content': '몽거는 양재시민의 숲에서. 도보로는 15분거리에 위치하고 있어! 주변에 이마트, 코스트코 등 큰 쇼핑센터가 있어서. 마을버스도 다니고 있어! 양재시민의숲역 카페에서 찾기 쉽지않은. 주차가능한 카페라서 차 가져오시는 분들은. 편하게 카페

In [191]:
# 동일한 세션아이디로 테스트
response = agent_with_chat_history.invoke(
    {
        "input": "OCR 테이블 파싱하는 방법을 알려주세요"
    },
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...
OCR (Optical Character Recognition)을 사용하여 테이블을 파싱하는 과정은 이미지나 스캔된 문서 내의 텍스트를 추출하고, 이를 구조화된 데이터 형태로 변환하는 작업을 포함합니다. 다음은 OCR을 사용하여 테이블을 파싱하는 기본적인 단계입니다:

1. **이미지 준비**: 파싱할 테이블이 포함된 이미지나 문서를 준비합니다. 이미지의 품질이 좋을수록 OCR의 정확도가 높아집니다.

2. **OCR 엔진 선택**: Tesseract, Google Vision API, Microsoft Azure OCR 등 다양한 OCR 엔진 중에서 선택합니다. 각 엔진마다 특성과 성능이 다르므로, 요구사항에 맞는 엔진을 선택하는 것이 중요합니다.

3. **텍스트 추출**: 선택한 OCR 엔진을 사용하여 이미지에서 텍스트를 추출합니다. 이 과정에서 엔진은 이미지 내의 문자를 인식하고, 이를 디지털 텍스트로 변환합니다.

4. **테이블 구조 인식**: 추출된 텍스트 중에서 테이블 구조를 인식합니다. 일부 OCR 엔진은 테이블 구조를 자동으로 인식하는 기능을 제공하지만, 그렇지 않은 경우에는 텍스트의 위치 정보를 사용하여 수동으로 테이블 구조를 재구성해야 할 수 있습니다.

5. **데이터 정제 및 변환**: 테이블 구조를 인식한 후, 필요에 따라 데이터를 정제하고 원하는 형식으로 변환합니다. 예를 들어, 불필요한 공백을 제거하거나, 특정 열의 데이터를 숫자 형태로 변환하는 등의 작업을 수행할 수 있습니다.

6. **데이터 저장**: 파싱된 테이블 데이터를 데이터베이스, CSV 파일, JSON 파일 등 원하는 형식으로 저장합니다.

OCR을 사용한 테이블 파싱은 복잡한 테이블 구조나 낮은 이미지 품질로 인해 어려움을 겪을 수 있습니다. 따라서, 고급 OCR 기술이나 추가적인 이미지 전처리 기술을 적용하여 정확도를 높일 수 있습니다.

실제 구현에 있어서는 사용하고자 하는 OCR 엔진의 

### **Agent Template Full Code** 

In [194]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/content/layout-parser-readthedocs-io-en-latest.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="PDF와 관련된 질문은 이 도구를 사용해야 합니다!",
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

########## 4. Agent 를 정의합니다 ##########

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# 채팅 메시지 기록을 관리하는 객체를 생성합니다.
message_history = ChatMessageHistory()

# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대부분의 실제 시나리오에서 세션 ID가 필요하기 때문에 이것이 필요합니다
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    lambda session_id: message_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. 질의-응답 테스트를 수행합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {
        "input": "OCR 테이블 파싱하는 방법을 알려주세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...
OCR (Optical Character Recognition)을 사용하여 테이블을 파싱하는 과정은 이미지나 PDF 내의 텍스트를 추출하고, 이를 구조화된 형태로 변환하는 작업을 포함합니다. 테이블 파싱은 특히 데이터를 정확하게 추출하고 구조화하는 것이 중요한 작업입니다. 다음은 OCR을 사용하여 테이블을 파싱하는 기본적인 단계입니다:

1. **이미지 준비**: 파싱할 테이블이 포함된 문서나 이미지를 준비합니다. 이미지의 품질이 좋을수록 OCR의 정확도가 높아집니다.

2. **OCR 엔진 선택**: Tesseract, Google Vision API, Amazon Textract 등 다양한 OCR 엔진 중에서 선택할 수 있습니다. 각 엔진마다 특성과 성능이 다르므로, 요구사항에 맞는 엔진을 선택하는 것이 중요합니다.

3. **텍스트 추출**: 선택한 OCR 엔진을 사용하여 이미지에서 텍스트를 추출합니다. 이 과정에서 엔진은 이미지 내의 문자를 인식하고, 이를 디지털 텍스트로 변환합니다.

4. **테이블 구조 인식**: 추출된 텍스트만으로는 테이블의 구조를 알 수 없기 때문에, 테이블의 행과 열을 인식하는 추가적인 처리가 필요합니다. 일부 OCR 엔진은 테이블 구조를 자동으로 인식하는 기능을 제공하기도 합니다.

5. **데이터 정제 및 변환**: 테이블 구조를 인식한 후, 데이터를 필요한 형태로 정제하고 변환합니다. 예를 들어, 특정 열만 추출하거나, 데이터를 CSV, JSON 등의 형식으로 변환할 수 있습니다.

6. **검증 및 수정**: OCR과 테이블 파싱 과정에서 오류가 발생할 수 있으므로, 결과 데이터를 검증하고 필요한 경우 수동으로 수정합니다.

### 예시: Tesseract를 사용한 테이블 파싱

Tesseract OCR은 오픈 소스 OCR 엔진으로, 테이블 파싱에도 사용할 수 있습니다. 다음은 Tesseract를 사용하여 테이블을 파싱하는 간단한 예시입니다:

1.